# Парсер квартир под сдачу 

## Подключение 

`rm -r venv`

`source venv/bin/activate`

`pip install -r requirements.txt`

`python -m ipykernel install --user --name=venv --display-name "Python (parser)"`

In [1]:
# Библиотеки для анализа данных
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

import datetime
import time

import os
import re

from tqdm import tqdm

# Библиотеки для парсинга 
import requests

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from webdriver_manager.chrome import ChromeDriverManager

# Старый парсер 
# from src.parser_avito import parse_avito_page

# Парсер 
from src.parser import parse_avito_page
from src.tools import get_ad_urls, get_photos


## Константы 

In [2]:
URL_TEMPLATE = 'https://www.avito.ru/sankt-peterburg/kvartiry/sdam-ASgBAgICAUSSA8gQ?context=&f=ASgBAQICAUSSA8gQAUDMCCSOWYxZ&p='
PAUSE_DURATION_SECONDS = 2
NUM_PAGES = 20

columns = [
    'title', 'price', 'characteristics', 
    'description', 'location', 'link', 'photo'
]

## Запуск сервиса

In [3]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [4]:
# !rm -rf ~/.wdm

## Запуск парсера

In [4]:
output_folder = "data/folder5"
os.makedirs(output_folder, exist_ok=True)

# Получаем список всех файлов в папке
files = os.listdir(output_folder)

# Ищем файлы с расширением .csv и извлекаем числовые части
numbers = []
for file in files:
    if file.endswith('.parquet'):
        match = re.search(r'\d+', file)
        if match:
            numbers.append(int(match.group()))

# Находим максимальное число
max_number = max(numbers) if numbers else None
print(f"Максимальный номер: {max_number}")


if not max_number: 
    start_page_number = 1
else: 
    start_page_number = max_number + 1

end_page_number = start_page_number + NUM_PAGES

print(f"Начнем парсинг со страницы номер: {start_page_number} (вкл)")
print(f"Закончим парсинг на странице номер: {end_page_number} (не вкл)")

Максимальный номер: 10
Начнем парсинг со страницы номер: 11 (вкл)
Закончим парсинг на странице номер: 31 (не вкл)


In [5]:
print(output_folder)

for page_num in range(start_page_number, end_page_number):
    
    print(f"Начали парсинг страницы # {page_num}")

    df_page = pd.DataFrame(columns=columns)
    
    try:
        # Загрузка страницы с объявлениями
        url = URL_TEMPLATE + str(page_num)
        driver.get(url)
        time.sleep(PAUSE_DURATION_SECONDS)

        # Получаем HTML-код страницы
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
  
        # Извлекаем ссылки на объявления на текущей странице
        links = get_ad_urls(soup)

        for link in tqdm(links):
            
            try:
                # Переход на страницу объявления
                driver.get(link)
                time.sleep(PAUSE_DURATION_SECONDS)  # Задержка для полной загрузки

                # Парсим данные на странице объявления (название, цена, фото, описание и т.д.)
                ad_data = parse_avito_page(driver=driver) # <- словарик 
                ad_data['link'] = link
                ad_data['photo'] = get_photos(driver=driver, cnt=20)

                df_page = pd.concat([df_page, pd.DataFrame([ad_data])], ignore_index=True)

                
            except TimeoutException:
                print(f"Ошибка: объявление {link} не загрузилось, пропускаем...")
                continue
            except Exception as e:
                print(f"Произошла ошибка при обработке объявления {link}: {e}")
                continue

        # Сохраняем данные для текущей страницы в DataFrame
        df_page.to_parquet(f'{output_folder}/cars_data_page_{page_num}.parquet')

    except TimeoutException:
        print(f"Ошибка: страница {page_num} не загрузилась, пропускаем...")
        continue
    except Exception as e:
        print(f"Произошла ошибка на странице {page_num}: {e}")
        continue



data/folder5
Начали парсинг страницы # 11
Произошла ошибка на странице 11: HTTPConnectionPool(host='localhost', port=57719): Read timed out. (read timeout=120)
Начали парсинг страницы # 12


 86%|████████████  | 43/50 [32:30<05:31, 47.41s/it]

Название не найдено
Цена не найдена
Характеристики не найдены
Описание не найдено
Расположение не найдено
Фото не найдено на этом этапе.


 88%|████████████▎ | 44/50 [33:13<04:36, 46.07s/it]

Ошибка при нажатии на кнопку: Message: 
Stacktrace:
0   chromedriver                        0x00000001033c3184 cxxbridge1$str$ptr + 3626716
1   chromedriver                        0x00000001033bb9d4 cxxbridge1$str$ptr + 3596076
2   chromedriver                        0x0000000102e28968 cxxbridge1$string$len + 89228
3   chromedriver                        0x0000000102e6cd50 cxxbridge1$string$len + 368756
4   chromedriver                        0x0000000102ea65b4 cxxbridge1$string$len + 604376
5   chromedriver                        0x0000000102e61568 cxxbridge1$string$len + 321676
6   chromedriver                        0x0000000102e621b8 cxxbridge1$string$len + 324828
7   chromedriver                        0x000000010338e9ac cxxbridge1$str$ptr + 3411716
8   chromedriver                        0x0000000103391ccc cxxbridge1$str$ptr + 3424804
9   chromedriver                        0x000000010337586c cxxbridge1$str$ptr + 3308996
10  chromedriver                        0x000000010339258c 

...

100%|██████████████| 50/50 [42:11<00:00, 50.63s/it]


Начали парсинг страницы # 30


 88%|████████████▎ | 44/50 [34:51<04:54, 49.12s/it]

Описание не найдено


100%|██████████████| 50/50 [38:54<00:00, 46.70s/it]


## Завершаем сессию

In [6]:
# Закрываем драйвер после завершения
driver.quit()